In [4]:
import numpy as np
import scipy as sp
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle 
from datetime import datetime
from os import walk
%load_ext Cython
import copy

In [5]:
import sys
sys.path.append("..")

In [6]:
import itertools
import ccxt
import enum

class BidAsk(enum.Enum):
    bid = 1
    ask = 2
class Order:
    def __init__(self, bid_ask, price, size):
        self.bid_ask = bid_ask
        self.price = price
        self.size = size
        # 念のため
        assert isinstance(bid_ask, BidAsk), "bid_ask is not instance of BidAsk"
        assert isinstance(price, float), "Price is not type of float"
        assert isinstance(size, float), "Size is not type of float"
class OrderBook:
    def __init__(self, exchange_id, bids, asks):
        self.exchange_id = exchange_id
        self.bids = bids
        self.asks = asks

    def best_ask(self):
        return self.asks[0]

    def best_bid(self):
        return self.bids[0]
class Position:
    def __init__(self, exchange_id, position_id, price, size, hasShort=False):
        self.exchange_id = exchange_id
        self.position_id = position_id
        self.price = price
        self.size = size
        self.hasShort = hasShort
class CrossPositions:
    def __init__(self, longPosition, shortPosition, spread, spreadTarget):
        self.spreadTarget = spreadTarget
        self.longPosition = longPosition
        self.shortPosition = shortPosition
        self.spread = spread

HOUR = 3
MAX_LENGTH = 60 * 60 * HOUR
import functools
# average_spread_dict = {}
# spread_dict = {}
profit = 0

In [7]:
def initExchangePairDict(array, init_value):
    exchange_pairs = list(itertools.permutations(array, 2))
    dict = {}
    for pair in exchange_pairs:
        key = pair[0] + '_' + pair[1]
        init = copy.deepcopy(init_value)
        dict[key] = init
    return dict

In [8]:
df_prices = pd.read_csv("coincheck_zaif_bitflyer3.csv")
# spread 計算
df_prices["coincheck_zaif"] = df_prices["coincheck_bid"] - df_prices["zaif_ask"]
df_prices["coincheck_bitflyer"] = df_prices["coincheck_bid"] - df_prices["bitflyer_ask"]
df_prices["zaif_coincheck"] = df_prices["zaif_bid"] - df_prices["coincheck_ask"]
df_prices["zaif_bitflyer"] = df_prices["zaif_bid"] - df_prices["bitflyer_ask"]
df_prices["bitflyer_coincheck"] = df_prices["bitflyer_bid"] - df_prices["coincheck_ask"]
df_prices["bitflyer_zaif"] = df_prices["bitflyer_bid"] - df_prices["zaif_ask"]

In [9]:
len(df_prices)

149704

In [10]:
simulation_summary = {}
def simulation(entry_top_percent, exit_top_percent, N):
    # N=100
    # sigma = 2
    profit = 0
    entry_exit_log_list = []
    entry_count = 0
    exit_count = 0
    # spread_keys=["coincheck_zaif","coincheck_bitflyer","zaif_coincheck","zaif_bitflyer","bitflyer_coincheck","bitflyer_zaif"]
    position_size = 0.01
    budget = 1000000 # 予算100万円
    clients = [
            ccxt.coincheck(),
            ccxt.zaif(),
            ccxt.bitflyer()
    ]
    exchanges = list(map(lambda ex: ex.id, clients))
    spread_dict = initExchangePairDict(exchanges, [])
    average_spread_dict = initExchangePairDict(exchanges, 0)
    standard_deviation_spread_dict = initExchangePairDict(exchanges, 0)
    myCrossPositions = initExchangePairDict(exchanges, [])

    index_count = 130000
    is_entry = [False] * index_count
    spread_keys=["coincheck_bitflyer","bitflyer_coincheck"]
    
    for index, row in df_prices.head(index_count).iterrows():
#         print(index)
        for pair_key in spread_keys:
#             print(index,pair_key)
#             print(index, N)
            if index>N:                
                past_N_rows = df_prices[index-N:index][pair_key]
                std = past_N_rows.std()
                mean = past_N_rows.mean()

                
                entry_threshold = past_N_rows.nlargest(int(len(past_N_rows)*(entry_top_percent))).min()
                exit_threshold = past_N_rows.nlargest(int(len(past_N_rows)*(exit_top_percent))).min()

                short_ex, long_ex = pair_key.split("_")
                ask_price = long_ex + "_ask"
                bid_price = short_ex + "_bid"
                    
                before_row = df_prices.iloc[index-1]
                conditions = is_entry[index - 1] == True and (row[ask_price] == before_row[ask_price] or row[bid_price] == before_row[bid_price])    
#                 print("before entry")
                if not conditions and row[pair_key] > 0 and row[pair_key] > entry_threshold:
#                     print("entry")
#                     print(row[pair_key], top_threshold)
                    is_entry[index] = True
                    entry_count+=1

                    longPosition = Position(long_ex, "dummyid", row[ask_price], position_size, False)
                    shortPosition = Position(short_ex, "dummyid", row[bid_price], position_size, True)
                    crossPosition = CrossPositions(longPosition, shortPosition, row[pair_key], exit_threshold) #meanはtarget spread
                    my_posi_dict_key = long_ex + '_' + short_ex
                    myCrossPositions[my_posi_dict_key].append(crossPosition)
                    entry_exit_log_list.append([
                        row.datetime,
                        "entry" ,
                        pair_key ,
                        row[ask_price],
                        row[bid_price],
                        row[pair_key],
                        0       
                    ])
                
                if len(myCrossPositions[pair_key]) > 0 and abs(row[pair_key]) < myCrossPositions[pair_key][-1].spreadTarget:
    #                   print("exit")
                    exit_count += 1
                    entry_position = myCrossPositions[pair_key].pop()
                    longProfit = (row[bid_price] - entry_position.longPosition.price) * position_size
                    shortProfit = (entry_position.shortPosition.price - row[ask_price]) * position_size
                    fee = entry_position.shortPosition.price * position_size * 0.0004
                    profit +=  longProfit + shortProfit - fee

                    entry_exit_log_list.append([
                        row.datetime,
                        "exit",
                        pair_key ,
                        row[ask_price],
                        row[bid_price],
                        row[pair_key],
                        profit                 
                    ])
    print(profit)
    remain_profit = 0 
    # exitできていないものがあればその時の値段でexit
    row_last = df_prices.iloc[index_count+1]
    for pair_key in spread_keys:
        short_ex, long_ex = pair_key.split("_")
        ask_price = long_ex + "_ask"
        bid_price = short_ex + "_bid"
        print("残り",len(myCrossPositions[pair_key]))
        if len(myCrossPositions[pair_key]) > 1:
            for remainCrossPosition in myCrossPositions[pair_key]:
                # entry数 - exit数
                longProfit = (row_last[bid_price] - remainCrossPosition.longPosition.price) * position_size
                shortProfit = (remainCrossPosition.shortPosition.price - row_last[ask_price]) * position_size
#                 print("row_last[bid_price]:",row_last[bid_price], "remainCrossPosition.longPosition.price", remainCrossPosition.longPosition.price)
#                 print("row_last[ask_price]",row_last[ask_price],"remainCrossPosition.shortPosition.price",remainCrossPosition.shortPosition.price)
#                 print(longProfit, shortProfit, profit)
                fee = remainCrossPosition.shortPosition.price * position_size * 0.0004
                remain_profit += longProfit + shortProfit - fee

                entry_exit_log_list.append([
                        row_last.datetime,
                        "exit",
                        pair_key ,
                        row_last[ask_price],
                        row_last[bid_price],
                        row_last[pair_key],
                        profit                 
                    ])

            print(remain_profit)
    
    profit += remain_profit
    
    with open("entry_exit_log_list_"+str(entry_top_percent)+"_"+str(exit_top_percent)+"_"+str(N)+".pickle", mode='wb') as f:
        pickle.dump(entry_exit_log_list, f)

    return [profit,remain_profit, entry_count, exit_count]

In [ ]:
# 13万件
simulation_summary_hukumieki = {}
for entry_k in [0.1,0.2,0.3,0.4,0.5,0.6]:
    for exit_k in [0.4,0.5,0.6,0.7]:
        print(entry_k,exit_k)
        summary=simulation(entry_k, exit_k,3600)
        print(summary)
        simulation_summary_hukumieki[(entry_k,exit_k)] = summary
        
with open("simulation_summary_hukumieki.pickle", mode='wb') as f:
    pickle.dump(simulation_summary_hukumieki, f)

0.1 0.4


In [22]:
simulation_summary_hukumieki

{(0.1, 0.4): [76313.58474799995, -12585.132640000005, 11643, 10401],
 (0.1, 0.5): [87225.104747999882, -12680.203072000006, 11643, 10316],
 (0.1, 0.6): [100796.10474799952, -12759.033760000009, 11643, 10277],
 (0.1, 0.7): [111901.02474799965, -12089.64529600001, 11643, 9976],
 (0.2, 0.4): [108233.92986799942, -15552.046644000004, 17444, 15944],
 (0.2, 0.5): [130177.52986800048, -16126.535984000015, 17444, 15782],
 (0.2, 0.6): [149281.92986800103, -16079.108424000036, 17444, 15310],
 (0.2, 0.7): [170235.07986799988, -12301.188160000052, 17444, 14762],
 (0.3, 0.4): [126327.62958800003, -18620.691995999994, 22530, 20847],
 (0.3, 0.5): [157405.24958800041, -20069.855379999983, 22530, 20357],
 (0.3, 0.6): [190082.87958800019, -19988.291575999985, 22530, 19645],
 (0.3, 0.7): [211908.45958800026, -17387.270608000024, 22530, 18921],
 (0.4, 0.4): [135660.41156799928, -20424.876959999987, 27308, 25469],
 (0.4, 0.5): [179704.76156799833, -23541.869715999987, 27308, 24546],
 (0.4, 0.6): [216976.57

In [20]:
import csv


with open('simulation_summary_hukumieki.csv', 'w') as f:  # Just use 'w' mode in 3.x
    w = csv.writer(f)
    w.writerows(simulation_summary_hukumieki.items())
#     w.writerow(simulation_summary_hukumieki)